In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

In [ ]:
import pandas as pd

df = pd.read_csv('D:\\Project\\DeepLearning\\proj3\\data\\train.csv')
print(df.columns)

In [ ]:
# ==============================================
# D:\Project\DeepLearning\proj3\src\engine\train.py
# ==============================================
import os,sys
import torch
import timm
import tqdm
import pathlib
from torch.utils.data import DataLoader

# ---------- 1. 以“本文件”为锚点，一次性算出根目录 ----------
FILE_ROOT = pathlib.Path(__file__).resolve()          # 当前文件
SRC_DIR   = FILE_ROOT.parent                          # ...\src\engine
PROJ_ROOT = SRC_DIR.parent.parent                     # ...\proj3

# ---------- 2. 所有路径全部用 Path，拼出来 ----------
CSV_TRAIN = PROJ_ROOT / 'data' / 'train.csv'
CSV_VAL   = PROJ_ROOT / 'data' / 'val.csv'
IMG_DIR   = PROJ_ROOT / 'data' / 'images'
CKPT_DIR  = PROJ_ROOT / 'experiments' / 'vit_baseline'

# 如果 experiments 下还有子目录，也一次性建好
os.makedirs(CKPT_DIR, exist_ok=True)

# ---------- 3. 把项目根目录加入 PYTHONPATH，避免 import 报错 ----------
sys.path.insert(0, str(PROJ_ROOT))

# ---------- 4. 正常 import 你自己的模块 ----------
from src.data.dataset import FundusDataset
from src.models.vit import get_model
from src.losses import FocalBCELoss
from src.metrics import MetricTracker
from src.utils.seed import seed_everything

# ===============================
# 超参数
# ===============================
cfg = {
    # === data ===
    "img_size": 224,
    "batch_size": 32,
    "num_workers": 8,
    "n_classes": 20,

    # === model ===
    "arch": "vit_base_patch16_224",
    "pretrained": True,
    "drop_rate": 0.1,

    # === optimizer ===
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "scheduler": "cosine",
    "warmup_epochs": 5,
    "epochs": 30,

    # === loss ===
    "loss_name": "focal",
    "focal_gamma": 2.0,
    "focal_alpha": None,

    # === misc ===
    "seed": 23,
    "ckpt_dir": CKPT_DIR,          # 直接用上面算好的路径
    "resume": None
}

# ===============================
# 训练主函数
# ===============================
def run(cfg):
    seed_everything(cfg['seed'])
    os.makedirs(cfg['ckpt_dir'], exist_ok=True)

    # ---- 数据 ----
    train_set = FundusDataset(CSV_TRAIN, IMG_DIR, img_size=224, mode='train')
    val_set   = FundusDataset(CSV_VAL,   IMG_DIR, img_size=224, mode='val')
    train_loader = DataLoader(train_set, cfg['batch_size'], shuffle=True,
                              num_workers=cfg['num_workers'], pin_memory=True)
    val_loader   = DataLoader(val_set,   cfg['batch_size'], shuffle=False,
                              num_workers=cfg['num_workers'], pin_memory=True)

    # ---- 模型 ----
    model = get_model(cfg['arch'], cfg['n_classes'],
                      pretrained=cfg['pretrained'],
                      drop_rate=cfg['drop_rate']).cuda()

    # ---- 损失 ----
    if cfg['loss_name'].lower() == 'focal':
        criterion = FocalBCELoss(alpha=None, gamma=cfg['focal_gamma']).cuda()
    else:
        criterion = torch.nn.CrossEntropyLoss().cuda()

    # ---- 优化器 & 调度器 ----
    optimizer = torch.optim.AdamW(model.parameters(),
                                  lr=cfg['lr'],
                                  weight_decay=cfg['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=cfg['epochs'])

    # ---- 训练循环 ----
    best_auc = 0.
    for epoch in range(1, cfg['epochs'] + 1):
        # --- train ---
        model.train()
        for x, y in tqdm.tqdm(train_loader, desc=f'Epoch {epoch} [Train]'):
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()
        scheduler.step()

        # --- val ---
        model.eval()
        metric = MetricTracker(cfg['n_classes'])
        with torch.no_grad():
            for x, y in tqdm.tqdm(val_loader, desc=f'Epoch {epoch} [Val]'):
                x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
                logits = model(x)
                metric.update(logits, y)
        val_dict = metric.compute()
        print(f'Epoch {epoch:02d} | val_auc={val_dict["auc"]:.4f}')

        # --- save best ---
        if val_dict['auc'] > best_auc:
            best_auc = val_dict['auc']
            save_path = cfg['ckpt_dir'] / 'best_auc.pth'
            torch.save(model.state_dict(), save_path)
            print(f'✅ Saved new best model (AUC={best_auc:.4f}) -> {save_path}')

# ---------- 启动 ----------
if __name__ == '__main__':
    run(cfg)

In [2]:
import os
from pathlib import Path
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from tqdm import tqdm

# -----------------------------
# Dataset
# -----------------------------
import pandas as pd
class FundusDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, img_dir, img_size=224, mode='train'):
        self.df = pd.read_csv(csv_file)
        self.img_dir = Path(img_dir)
        self.img_size = img_size
        self.mode = mode

        self.label_cols = self.df.columns[1:]  # 第一列 path，其余列为标签

        if mode == 'train':
            self.aug = A.Compose([
                A.Resize(img_size, img_size),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
                A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
                ToTensorV2()
            ])
        else:
            self.aug = A.Compose([
                A.Resize(img_size, img_size),
                A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['path']  # 根据你的列名
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.aug(image=image)['image']

        if self.mode in ['train', 'val']:
            label = torch.tensor(row[self.label_cols].astype(float).values, dtype=torch.float32)
            return image, label
        else:
            return image, row['path']


# -----------------------------
# Focal BCE Loss
# -----------------------------
class FocalBCELoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = torch.tensor(alpha, dtype=torch.float32) if alpha is not None else None
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, target):
        bce_loss = nn.functional.binary_cross_entropy_with_logits(logits, target, reduction='none')
        p = torch.sigmoid(logits)
        pt = p * target + (1 - p) * (1 - target)
        focal_factor = (1 - pt) ** self.gamma

        if self.alpha is not None:
            alpha_factor = self.alpha * target + (1 - self.alpha) * (1 - target)
            loss = alpha_factor * focal_factor * bce_loss
        else:
            loss = focal_factor * bce_loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# -----------------------------
# Model
# -----------------------------
import timm
def get_model(arch='vit_base_patch16_224', n_classes=20, pretrained=True, drop_rate=0.1):
    model = timm.create_model(arch, pretrained=pretrained, num_classes=n_classes, drop_rate=drop_rate)
    return model

# -----------------------------
# Metrics
# -----------------------------
def compute_metrics(y_true, y_pred):
    y_pred_label = (torch.sigmoid(y_pred) > 0.5).float()
    auc = roc_auc_score(y_true.cpu().numpy(), torch.sigmoid(y_pred).cpu().numpy(), average='macro')
    f1 = f1_score(y_true.cpu().numpy(), y_pred_label.cpu().numpy(), average='macro')
    acc = accuracy_score(y_true.cpu().numpy(), y_pred_label.cpu().numpy())
    return {'auc': auc, 'f1': f1, 'acc': acc}

# -----------------------------
# Training loop
# -----------------------------
def train(cfg):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    os.makedirs(cfg['ckpt_dir'], exist_ok=True)

    train_set = FundusDataset(cfg['train_csv'], cfg['img_dir'], cfg['img_size'], mode='train')
    val_set   = FundusDataset(cfg['val_csv'],   cfg['img_dir'], cfg['img_size'], mode='val')
    train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=cfg['num_workers'])
    val_loader   = DataLoader(val_set,   batch_size=cfg['batch_size'], shuffle=False, num_workers=cfg['num_workers'])

    model = get_model(cfg['arch'], cfg['n_classes'], cfg['pretrained'], cfg['drop_rate']).to(device)
    criterion = FocalBCELoss(gamma=cfg['focal_gamma']).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg['epochs'])

    best_auc = 0
    for epoch in range(1, cfg['epochs']+1):
        # ===== train =====
        model.train()
        train_loss = 0
        for x, y in tqdm(train_loader, desc=f'Epoch {epoch} [Train]'):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)
        train_loss /= len(train_loader.dataset)

        # ===== val =====
        model.eval()
        val_loss = 0
        y_true_list = []
        y_pred_list = []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = criterion(logits, y)
                val_loss += loss.item() * x.size(0)
                y_true_list.append(y)
                y_pred_list.append(logits)
        val_loss /= len(val_loader.dataset)
        y_true = torch.cat(y_true_list, dim=0)
        y_pred = torch.cat(y_pred_list, dim=0)
        metrics = compute_metrics(y_true, y_pred)

        print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, "
              f"val_auc={metrics['auc']:.4f}, val_f1={metrics['f1']:.4f}, val_acc={metrics['acc']:.4f}")

        # ===== save best model =====
        if metrics['auc'] > best_auc:
            best_auc = metrics['auc']
            torch.save(model.state_dict(), os.path.join(cfg['ckpt_dir'], 'best_model.pth'))
            print(f"Saved best model at epoch {epoch} with AUC={best_auc:.4f}")

# -----------------------------
# Config
# -----------------------------
cfg = {
    'train_csv': 'D:\\Project\\DeepLearning\\proj3\\data\\train.csv',
    'val_csv': 'D:\\Project\\DeepLearning\\proj3\\data\\val.csv',
    'img_dir': 'D:\\Project\\DeepLearning\\proj3\\data\\images',
    'img_size': 224,
    'batch_size': 32,
    'num_workers': 0,
    'n_classes': 20,
    'arch': 'vit_base_patch16_224',
    'pretrained': True,
    'drop_rate': 0.1,
    'lr': 3e-4,
    'weight_decay': 1e-4,
    'epochs': 30,
    'focal_gamma': 2.0,
    'ckpt_dir': 'experiments/vit_bce_focal'
}

# -----------------------------
# Run
# -----------------------------
if __name__ == '__main__':
    train(cfg)


Epoch 1 [Train]:   0%|          | 0/56 [00:00<?, ?it/s]c:\Users\Yqj\anaconda3\envs\new_env\lib\site-packages\timm\layers\attention.py:80: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(
Epoch 1 [Train]:  11%|█         | 6/56 [01:33<12:57, 15.55s/it]


KeyboardInterrupt: 